In [1]:
library(ggplot2)
library(dplyr)
library(purrr)
library(Seurat)
library(viridis)
library("RColorBrewer")
source("../Scripts/functions.R")

getName <- function(folder, position){
  name <- unlist(stringr::str_split(folder,"/"))
  name <- name[length(name)]
  name <- unlist(stringr::str_split(name,"_"))[position]
  return(name)
}

getVector <- function(method, folder){
    if(method %in% c("Seurat", "SingleCellNet", "CellID", "SingleR")){
  if(stringr::str_detect(folder, "predict")) stop("Wrong file")
  name <- paste(sep="_", getName(folder,1), method ,getName(folder,2),
                stringr::str_replace(getName(folder,3), ".txt", "") )
  df <- read.csv(folder, sep="\t")
  df <- df[,c("id", "predicted")] #.match
  colnames(df) <- c("id", name)
    
} else if (method == "ItClust"){
  name <- paste(sep="_",  getName(folder,1), "ItClust",getName(folder,2),
                getName(folder,3))
  df <- read.csv(paste(folder, "results.txt", sep="/"), sep="\t")
  df <- df[,c("class_", "predicted_celltype", "cell_id")]
  colnames(df) <- c("class", "predicted", "id")
  df[,name] <- df$predicted 
  df <- df[, c("id",  name)]
}
    rownames(df) <- df$id
    return(df)
}

get_results_method <- function(resultfolder, id, method){
    print(paste("Start", method, "..."))
    files <- list.files(resultfolder, pattern=id, full.names = T)
    if (method == "Seurat") files <- files[stringr::str_detect(files,
                                                               "predict", negate=T)]
    data <- lapply(files, function(file) getVector(method, file))
    summary <- data %>% reduce(full_join, by = "id")
    return(summary)
}
                   
translate <- function(col){
  
    col[col == "B"] <- "B cell"
    col[col == "Cytotoxic T"] <- "Cytotoxic T cell"
    col[col == "CD4+ T"] <- "CD4+ T cell"
    col[col == "Dendritic"] <- "Dendritic cell"
    col[col == "Natural killer"] <- "Natural killer cell"
    col[col == "Plasmacytoid dendritic"] <- "Plasmacytoid dendritic cell"
    return(col)
} 
                   
adjust_names <- function(data, name="ItClust"){
    cols <- colnames(data)[stringr::str_detect(colnames(data), name)] 
    x<- do.call(cbind,lapply(cols, function(col) translate(data[,col])))
    colnames(x) <- cols  
    id <- data[,!(colnames(data) %in% cols)]

    data <- cbind(x,id)
    return(as.data.frame(data))
}
                             
transform_PBMC_results <- function(data, celltypes, methods, sizes,
                                   cols= c("id","nGene", "nUMI", "percent.mito",
                                           "Cluster", "class_", "Experiment", "Method")){
    #data <- read.csv(data_file)
    x <- reshape2::melt(data,  id.vars =cols,value.name = "Prediction")
    
    x[c('Reference', 'Approach', "Size", "Set")] <- stringr::str_split_fixed(x$variable, '_', 4)
    x$Match <- x$Prediction == x$class_ 

    x$refSize <- sizes[ match(x$class_, celltypes ) ]
    x$Approach <- factor(x$Approach, levels=methods)
    x$class <- factor(x$class_, levels=celltypes)
    x <- x[,c("id", "Prediction", "Reference", "Approach", "Size", "Set",
              "class", "refSize", "Match")]
    x$Size <- as.numeric(x$Size)
    x <- x[!is.na(x$Match),]
    return(x)
}  
                             
read_data_results <- function(file, path, method){
    data <- read.csv(paste(path,file, sep="/"), sep="\t")
    data$tag <- stringr::str_replace(file, ".txt", "")
    data$method <- method
    data <- data[, c("id", "predicted", "tag", "method")]
    return(data)
}
get_data <- function(path, method){
   seurat_files <- list.files(path, full.names = F)
   if(method =="Seurat") seurat_files <- seurat_files[stringr::str_detect(seurat_files, "pred",
                                                                          negate = T)]
   seurat <- do.call(rbind, lapply(seurat_files,
                                   function(file) read_data_results(file, path, method)))
   return(seurat)
 
}
get_measures_set <- function(data, type, method,set){
    
    data <- data[ data$method == method & data$set == set,]
    tp <- length(data$predicted[data$predicted == type & data$class_ == type])
    fp <- length(data$predicted[data$predicted == type & data$class_ != type])
    fn <- length(data$predicted[data$predicted != type & data$class_ == type])
    tn <- length(data$predicted[data$predicted != type & data$class_ != type])
    precision <- tp / (tp + fp)
    recall <- tp / (tp + fn)
    f1 <- 2*(precision * recall) / (precision + recall)
    accuracy <- (tp) / length(data$predicted[data$class_ == type])
    return(data.frame("class"=type,"method"=method,"set"=set,
                      "precision"=precision,"recall"=recall,"f1"=f1, "accuracy"=accuracy))
}
get_predictions <- function(path,ref){
    seurat <- get_data(paste(sep="/", path, "Seurat"), "Seurat")
    singler <- get_data(paste(sep="/", path,"SingleR/"), "SingleR")
    scn <- get_data(paste(sep="/", path,"SingleCellNet/"), "SingleCellNet")
    cellid <- get_data(paste(sep="/", path,"/CellID/"), "CellID")
    itclust_files <- list.files(paste(sep="/", path,"ItClust"), full.names = F)
    itclust <- do.call(rbind, lapply(itclust_files,
                                     function(file) getItClust(file,
                                                               paste(sep="/", path,"ItClust"),
                                                               "ItClust")))
    itclust$predicted <-   translate(itclust$predicted)                        
    data <- do.call(rbind,list(itclust, seurat, cellid, singler, scn))
    data[c('reference', 'size', "set")] <- stringr::str_split_fixed(data$tag, '_', 3)                                
    predictions <-  weighted_bootstrap(read.csv(ref), data)
    return(predictions)
}
getItClust <- function(file, path, method){
    data <- read.csv(paste(path,file, "results.txt", sep="/"), sep="\t")
    data$tag <- stringr::str_replace(file, ".txt", "")
    data$method <- method
    data <- data[, c("cell_id", "predicted_celltype", "tag", "method")]
    colnames(data) <- c("id", "predicted", "tag", "method")
    return(data)
}
translate <- function(col){
  
    col[col == "B"] <- "B cell"
    col[col == "Cytotoxic T"] <- "Cytotoxic T cell"
    col[col == "CD4+ T"] <- "CD4+ T cell"
    col[col == "Dendritic"] <- "Dendritic cell"
    col[col == "Natural killer"] <- "Natural killer cell"
    col[col == "Plasmacytoid dendritic"] <- "Plasmacytoid dendritic cell"
    return(col)
} 
get_measures_id <- function(data){
    data$match <- data$class_ == data$predicted
    data$match[data$match == TRUE] <- 1
    summary <- data %>% 
           dplyr::group_by(id, class_, method) %>% 
           dplyr::summarize(score = sum(match) / n()) 
    
    return(summary)
}
weighted_bootstrap <- function(ref, data){
    n <- data.frame(table(ref$class_))
    data$score <- n$Freq[match( data$predicted, n$Var1)]
    data$score <- 1 / (abs(data$score - as.integer(data$size)) + 1)
    data <- data[data$set != 0,]
    
    summary <- data %>% 
           dplyr::group_by(id, predicted, set, method, reference) %>% 
           dplyr::summarize(score = sum(score)) 
    
    df.wide <- tidyr::pivot_wider(summary, names_from = predicted, values_from = score)
    df.wide[is.na(df.wide)] <- 0
    names <-  colnames(df.wide[, 5:ncol(df.wide)])
    df.wide$predicted <- apply(df.wide[, 5:ncol(df.wide)], 1, which.max)
    df.wide$predicted <- names[df.wide$predicted]
    df <- df.wide[, c("id", "set", "method", "predicted")] 
    return(df)
}
                                     
get_summary <- function(folder, meta, outfolder, query, name, celltypes, methods, seqs, full){
    data <- get_predictions(folder, meta)
    data <- merge(data, query, by="id")
    
    data_set <- do.call(rbind, lapply(celltypes, function(type) 
            do.call(rbind, lapply(methods,   function(method) 
            do.call(rbind, lapply(seqs,function(set) get_measures_set(mono, type,
                                                                             method, set)))))))
    data_set <-  merge(data_set, full, by=c("class", "method"))
    data_id <- get_measures_id(data) 
                                  
    write.table(mono_set, paste(sep="/", outfolder, paste0("summary_", name, ".csv" )),
                col.names=T, row.names=T, quote=T, sep=",")
    return(data_id)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching SeuratObject

Loading required package: viridisLite



In [2]:
celltypes = c("Cytotoxic T cell", "CD4+ T cell", "CD14+ monocyte", "B cell", "Megakaryocyte",
              "Natural killer cell", "CD16+ monocyte", "Dendritic cell",
              "Plasmacytoid dendritic cell")

methods <- c("Seurat", "SingleR","CellID", "SingleCellNet", "ItClust")  
sizes <- c(3090, 2418, 1373, 1022, 703, 623, 273, 126, 38)
names(sizes) <- celltypes


query <- read.csv("../Data/Fulldata/PBMC_Query/meta.csv")
folder <- "../Data/Predictions/"
name <- "PBMC10x"
#data <- read.csv("../Results/Files/result_general.csv")


In [ ]:
seurat <- get_results_method(paste(sep="/",folder,"Seurat"), name, "Seurat")
scn <- get_results_method(paste(sep="/",folder,"SingleCellNet" ), name, "SingleCellNet")
itclust <- get_results_method(paste(sep="/",folder,"ItClust" ), name, "ItClust")
itclust <- adjust_names(itclust)
singleR <- get_results_method(paste(sep="/",folder,"SingleR" ), name, "SingleR")
cellid <- get_results_method(paste(sep="/",folder,"CellID" ), name, "CellID")
                             
data <- list(cellid, seurat, scn, singleR) %>% reduce(full_join, by = "id")#, itclust
data <- merge(data,query)  
rownames(data) <- data$id
head(data)

[1] "Start Seurat ..."
[1] "Start SingleCellNet ..."


In [ ]:
#long <- transform_PBMC_results(data, celltypes, methods, sizes)  
#print(head(long))

full <- do.call(rbind, lapply(unique(long$class),
                                  function(type) do.call(rbind,lapply(unique(long$Approach),
                                  function(method)  get_measures(long, type,
                                                                               "PBMC10x", method,
                                                                               3090, 0)))))
print(head(full))

                                                                      
all <- long[long$Set <= 20 & long$Size %in% c(38,100,250, 500,1000,1500,2000,3000),]
all <- do.call(rbind, lapply(unique(long$class),
                                 function(type) do.call(rbind,lapply(unique(long$Approach),
                                  function(method) do.call(rbind, lapply(unique(long$Size),
                                  function(size) do.call(rbind, lapply(unique(long$Set),
                                  function (set) get_measures(all, type, "PBMC10x",
                                                             method, size,set)))))))))
                                                                       
print(head(all))                                                             
write.csv(all, "../Results/Files/values_all.csv")  
write.csv(full, "../Results/Files/values_full.csv") 

             class reference        method size set precision    recall
1           B cell   PBMC10x        CellID 3090   0 0.7975581 0.7592801
2           B cell   PBMC10x        Seurat 3090   0 0.9070175 0.9692538
3           B cell   PBMC10x SingleCellNet 3090   0 0.9224806 0.9370079
4           B cell   PBMC10x       SingleR 3090   0 0.9142091 0.8950131
5 Cytotoxic T cell   PBMC10x        CellID 3090   0 0.7325581 0.5280106
6 Cytotoxic T cell   PBMC10x        Seurat 3090   0 0.6714227 0.7265108
         f1  accuracy
1 0.7779485 0.7592801
2 0.9371035 0.9692538
3 0.9296875 0.9370079
4 0.9045093 0.8950131
5 0.6136888 0.5280106
6 0.6978814 0.7265108


In [9]:
full <- full[, c("class", "method", "precision", "recall", "f1", "accuracy")]
colnames(full) <-  c("class", "method", "full_precision", "full_recall", "full_f1",
                     "full_accuracy")
mono <- get_summary("../Data/Predictions_abundancebased//",
                    "../Data/Fulldata/PBMC10x_Reference/meta.csv",
                    "../Results/Files/", "mono", celltypes, methods, seq(1,20,1), full)

mosaic <- get_summary("../Data/Predictions_mosaic/",
                    "../Data/Fulldata/PBMC10x_Reference/meta.csv",
                    "../Results/Files/", "mosaic", celltypes, methods, seq(1,20,1), full)

ERROR: Error in `colnames<-`(`*tmp*`, value = c("class", "method", "full_precision", : attempt to set 'colnames' on an object with less than two dimensions


In [ ]:
full <- data[, stringr::str_detect(colnames(data), "3090")]
full$class_ <- data$class_
full$id <- data$id
full$tech <- data$Method
full <- reshape2::melt(full,id=c("class_", "id", "tech"), value.name = "predicted")
full$score <-  full$class_ == full$predicted
full$score[full$score == TRUE] <- 1
full[c('reference', 'method', "size", "set")] <- stringr::str_split_fixed(full$variable, '_', 4)
full <- full[, c("id", "class_", "method", "score", "predicted", "tech")]


In [ ]:
mosaic <- tidyr::pivot_wider(mosaic_id, names_from = c(method), values_from = score,
                             names_pref="mosaic_")
mono <- tidyr::pivot_wider(mono_id, names_from = c(method), values_from = score,
                             names_pref="mono_")

full1 <- tidyr::pivot_wider(full[, colnames(full) != "predicted"], names_from = c(method),
                            values_from = score,
                             names_pref="full_")

full2 <- tidyr::pivot_wider(full[, colnames(full) != "score"], names_from = c(method),
                            values_from = predicted,
                             names_pref="fullPred_")


data <- Reduce(function(x, y) merge(x, y, all=TRUE), list(mosaic, mono, full1, full2))
write.table(data, "../Results/Files/umap_data.csv", sep=",")